In [ ]:
import datasets  # to import Mozilla Common Voice and Librispeech dataset
import transformers #Huggingface Wav2Vec2 model loading
import torchaudio #Audioprocessing
import soundfile as sf #Audioprocessing 2.0[Backup]
import numpy as np
import jiwer
import librosa


Estabilishing Base Directory

In [ ]:
base_dir= "C:\\Users\\skdwi\\OneDrive\\Desktop\\ASR"

<p>ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, e.g. a feature vector, and a tokenizer that processes the model's output format to text.</p>

<p>In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called Wav2Vec2CTCTokenizer, and a feature extractor, called Wav2Vec2FeatureExtractor.</p>



1. Importing Dataset:- Using Timit in this case because it is small, ideally one must use Librispeech although training will take alot of time

In [ ]:
from datasets import load_dataset, load_metric
libri_train = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train+validation",trust_remote_code=True)
libri_test = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test",trust_remote_code=True)
#removing unneccesary outputs
libri_train = libri_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
libri_test = libri_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
#Also must remove special Characters-- It is important because special Characters have no sound, so must be stripped
# Apply the function to the dataset
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

libri_test= libri_test.map(remove_special_characters)
libri_train= libri_train.map(remove_special_characters)

print(libri_test)


<h3>Creating a Vocab.json</h3>
<p>Similar to identifying classes in Neural Networks, we also need to create a vocabulary of the english language here to map out the speech to text clearly. Helps in encoding decoding</p>

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])#Combines all the data in all the trascription
    vocab = list(set(all_text))#Sets don't allow duplicate, so only allows unique
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = libri_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=libri_test.column_names)
vocab_train = libri_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=libri_test.column_names)

combined_vocab = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {char: idx for idx, char in enumerate(sorted(combined_vocab))}#Create a dictionary

import json
with open('vocab.json', 'w') as f:
    json.dump(vocab_dict, f, indent=0)

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
print(libri_test)


<p>Finally, we also add a padding token that corresponds to CTC's "blank token". The "blank token" is a core component of the CTC algorithm.</p>



In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print(len(vocab_dict))
print(libri_test)



<p>We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.</p>
<p>Encoding is Basically changing the text to vocab id according to vocab.json</p>
<p>attention_mask: A sequence of 1s and 0s indicating which tokens should be attended to by the model. In this case, all values are 1 because there is no padding, meaning all tokens are valid and should be processed by the model.
</p>

<p>To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object.</p>

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,TrainingArguments, Trainer,Wav2Vec2ForCTC
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, requires_grad=False)
#the feature size of 1 indicates that the model processes a one-dimensional raw audio waveform--Amplitude

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
print(libri_test)

libri_train[0]

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

libri_train = libri_train.map(speech_file_to_array_fn)
libri_test = libri_test.map(speech_file_to_array_fn)

In [ ]:
print(libri_test)

def prepare_dataset(batch):
    try:
        # Access the pre-loaded audio data and reduce the sampling rate
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
        batch["sampling_rate"] = 16_000
        
        speech_array = batch["audio"]["array"]
        sampling_rate = batch["audio"]["sampling_rate"]
        
        # Process the audio data
        batch["input_values"] = processor(speech_array.squeeze().numpy(), padding="longest", sampling_rate=sampling_rate).input_values[0]
        batch["labels"] = processor(text=batch["sentence"], return_tensors="pt").input_ids[0]
    except Exception as e:
        print(f"Error processing batch: {e}")
    
    return batch

libri_test = libri_test.map(prepare_dataset, batch_size=8, num_proc=4, batched=True)
libri_test = libri_train.map(prepare_dataset, batch_size=8, num_proc=4, batched=True)

print(libri_test)


<h3>Data Collator</h3>

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer_metric = load_metric("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


<h3>Training Arguments and Training</h3>

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

training_args = TrainingArguments(
  output_dir="./wav2vec2-finetuned",  # Directory to save the model checkpoints and logs
  per_device_train_batch_size=32,     # Batch size per device during training
  eval_strategy="steps",              # Evaluation strategy to use (in this case, evaluate every few steps)
  num_train_epochs=3,                 # Number of epochs to train the model
  save_steps=500,                     # Save checkpoint every 500 steps
  eval_steps=500,                     # Evaluate the model every 500 steps
  logging_steps=100,                  # Log training metrics every 100 steps
  learning_rate=1e-4,                 # Learning rate for the optimizer
  save_total_limit=2,                 # Limit the total amount of checkpoints to save (older checkpoints will be deleted)
  remove_unused_columns=False,        # Do not remove unused columns
  group_by_length=True,
  fp16=True,
  gradient_checkpointing=True
)

print(libri_test.column_names)
print(libri_test[0]["sentence"])  

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=libri_train,
    eval_dataset=libri_test,
    tokenizer=processor.feature_extractor,
)


trainer.train()
